# 模型推論（預測新資料）

> 這個 notebook 用於載入已訓練好的模型，對新的胚胎照片進行預測

## 使用說明

1. 上傳訓練好的模型檔案 (`best_model.pth`)
2. 上傳要預測的胚胎照片（放在一個資料夾內）
3. 執行所有 cell
4. 下載預測結果 CSV 檔案

## Step 1: 安裝和載入套件

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import os
from google.colab import files
import zipfile
import shutil

# 檢查 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'使用裝置: {device}')

## Step 2: 上傳模型檔案

請上傳你從 EXP-001 訓練好的 `best_model.pth` 檔案

In [ ]:
# 上傳模型檔案
print("請上傳 best_model.pth 檔案...")
uploaded = files.upload()

# 確認上傳成功
model_path = list(uploaded.keys())[0]
print(f"\n✅ 模型檔案已上傳: {model_path}")

## Step 3: 載入模型

載入 EXP-001 使用的 ResNet18 模型

In [ ]:
# 建立與訓練時相同的模型架構
model = models.resnet18(weights=None)  # 不載入預訓練權重
model.fc = nn.Linear(model.fc.in_features, 2)  # 改成 2 類輸出

# 載入訓練好的權重
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()  # 設為評估模式

print("✅ 模型載入成功！")

## Step 4: 上傳要預測的照片

請將所有要預測的胚胎照片打包成 **zip 檔案**，然後上傳

In [ ]:
# 建立存放照片的資料夾
inference_dir = '/content/inference_images'
if os.path.exists(inference_dir):
    shutil.rmtree(inference_dir)
os.makedirs(inference_dir)

# 上傳 zip 檔案
print("請上傳包含胚胎照片的 zip 檔案...")
uploaded = files.upload()

# 解壓縮
zip_path = list(uploaded.keys())[0]
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(inference_dir)

# 找出所有圖片檔案（支援 jpg, jpeg, png）
image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
image_files = []

for root, dirs, filenames in os.walk(inference_dir):
    for filename in filenames:
        if any(filename.endswith(ext) for ext in image_extensions):
            image_files.append(os.path.join(root, filename))

print(f"\n✅ 找到 {len(image_files)} 張圖片")
print("\n圖片列表:")
for img in image_files[:10]:  # 只顯示前 10 張
    print(f"  - {os.path.basename(img)}")
if len(image_files) > 10:
    print(f"  ... 還有 {len(image_files) - 10} 張")

## Step 5: 設定圖片預處理

使用與訓練時相同的預處理方式

In [ ]:
# 定義圖片預處理（與訓練時相同）
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

print("✅ 圖片預處理設定完成")

## Step 6: 進行預測

In [ ]:
# 進行預測
results = []

print("開始預測...\n")

with torch.no_grad():
    for img_path in image_files:
        # 載入並預處理圖片
        image = Image.open(img_path).convert('RGB')
        image_tensor = transform(image).unsqueeze(0).to(device)
        
        # 預測
        outputs = model(image_tensor)
        probabilities = torch.softmax(outputs, dim=1)
        
        # 取得「會著床」的機率（Class 1）
        prob_implant = probabilities[0][1].item()
        
        # 預設閾值 0.5 的預測結果
        prediction = "會著床" if prob_implant >= 0.5 else "不會著床"
        
        results.append({
            '檔案名稱': os.path.basename(img_path),
            '著床機率': round(prob_implant, 4),
            '預測結果(閾值0.5)': prediction
        })

print(f"✅ 完成！共預測 {len(results)} 張圖片")

## Step 7: 查看預測結果

In [ ]:
# 建立結果表格
df_results = pd.DataFrame(results)

# 按著床機率排序（高到低）
df_results = df_results.sort_values('著床機率', ascending=False).reset_index(drop=True)

# 顯示結果
print("=" * 60)
print("預測結果（按著床機率由高到低排序）")
print("=" * 60)
display(df_results)

# 統計摘要
print("\n" + "=" * 60)
print("統計摘要")
print("=" * 60)
print(f"總共預測: {len(df_results)} 張")
print(f"預測『會著床』: {(df_results['預測結果(閾值0.5)'] == '會著床').sum()} 張")
print(f"預測『不會著床』: {(df_results['預測結果(閾值0.5)'] == '不會著床').sum()} 張")
print(f"\n著床機率統計:")
print(f"  平均: {df_results['著床機率'].mean():.4f}")
print(f"  最高: {df_results['著床機率'].max():.4f}")
print(f"  最低: {df_results['著床機率'].min():.4f}")

## Step 8: 下載預測結果

將結果匯出為 CSV 檔案

In [ ]:
# 儲存為 CSV
output_path = '/content/prediction_results.csv'
df_results.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ 結果已儲存為: {output_path}")

# 下載檔案
files.download(output_path)
print("\n📥 檔案已開始下載...")

---

## 進階功能：調整判斷閾值

如果你想用不同的閾值來判斷，可以執行下面的 cell

In [ ]:
# 嘗試不同閾值
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

print("不同閾值下的預測結果：\n")
for threshold in thresholds:
    count_implant = (df_results['著床機率'] >= threshold).sum()
    count_not_implant = len(df_results) - count_implant
    print(f"閾值 {threshold}: 預測『會著床』{count_implant} 張，『不會著床』{count_not_implant} 張")

In [ ]:
# 如果要用自訂閾值產生結果，修改下面的數值
custom_threshold = 0.4  # ← 修改這裡

df_results['預測結果(自訂閾值)'] = df_results['著床機率'].apply(
    lambda x: '會著床' if x >= custom_threshold else '不會著床'
)

print(f"使用閾值 {custom_threshold} 的結果：\n")
display(df_results)

# 儲存自訂閾值結果
output_path_custom = f'/content/prediction_results_threshold_{custom_threshold}.csv'
df_results.to_csv(output_path_custom, index=False, encoding='utf-8-sig')
files.download(output_path_custom)

---

## 備註

- 這個預測結果是基於 **EXP-001 (ResNet18)** 模型
- 模型的 Recall 為 76%，表示真正會著床的胚胎，AI 有 76% 機會猜對
- 模型的 Precision 為 50%，表示 AI 說「會著床」的，有 50% 是真的會著床
- **因為這批資料沒有實際標籤，所以只能得到預測結果，無法計算準確率**
- 等之後知道實際結果後，可以回來驗證模型預測的準確度